In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn import linear_model as lm

# Load Data

In [153]:
boston = load_boston()
X = boston.data 
X = preprocessing.normalize(X, norm='l2')
y = boston.target
X_train = X[0:300,:]
y_train = y[0:300]
X_test = X[300:,:]
y_test = y[300:]

In [154]:
p = np.random.rand(X_train.shape[1])
dp = np.diag(p) #B
dpp = np.diag(p-1.0/2*p*p)
num = X_test.shape[0] - p*X_test.shape[0]
num = num.astype('int32').tolist()
for i in range(len(num)): 
    ll = num[i]
    perm = np.random.permutation(X_test.shape[0])[0:ll]
    perm = perm.tolist()
    for j in perm:
        X_test[j,i] = 0
print 'finish loading data!'  

finish loading data!


# Ridge regression with L2

In [179]:
lam = 1000

In [180]:
inv = np.linalg.inv(np.dot(X_train.T,X_train)+lam*np.eye(X_train.shape[1]))
w_r = np.dot(np.dot(inv,X_train.T),y_train)
# error rate
dif = np.dot(X_test, w_r) - y_test
np.sum(np.power(dif,2))

69968.932506381287

In [156]:
w_r

array([  4.33673415e-03,   2.30534838e-01,   9.06760297e-02,
         1.20074606e-03,   5.98183029e-03,   8.02142178e-02,
         7.00547161e-01,   5.21160070e-02,   5.34993310e-02,
         3.56108486e+00,   2.04682467e-01,   4.56748648e+00,
         1.04078965e-01])

# Lasso Regression

In [157]:
clf = lm.Lasso(alpha=0.1)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
# error
dif = y_pred - y_test
np.sum(np.power(dif,2))

46980.244439960225

In [158]:
clf.coef_

array([ -0.        ,  13.78655243,  -0.        ,   0.        ,
        -0.        ,   0.        ,  -0.        ,  -0.        ,
         0.        , -18.84166474,  -0.        ,   0.        ,  -0.        ])

# Anytime with L1

In [159]:
lam = 0
xp = np.dot(X_train,dp)
inverse = np.linalg.inv(np.dot(np.dot(X_train.T,X_train),dpp) + 1.0/2*np.dot(xp.T,xp)+lam*np.eye(X_train.shape[1]))
w_p = np.dot(np.dot(inverse,xp.T),y_train)

In [160]:
w_p

array([ -3.02803390e+04,   1.08068482e+02,  -3.59011018e+02,
         2.51037151e+03,   6.29010101e+04,   3.35288389e+03,
        -3.92978345e+01,  -1.87791649e+01,   8.99588101e+01,
        -1.28635529e+01,   1.66597404e+03,  -2.35855115e+03,
        -5.66902793e+02])

In [161]:
np.max(w_p)

62901.010090485579

In [174]:
thr = 30000

In [175]:
w_p_list = w_p.tolist()
for i in range(len(w_p_list)):
    it = w_p_list[i]
    if it > thr:
        it -= thr
    elif it <= thr and it >= -thr:
        it = 0
    elif it < -thr:
        it += thr
    w_p_list[i] = it
w_p_list = np.asarray(w_p_list)
w_p_list

array([  -280.33899175,      0.        ,      0.        ,      0.        ,
        32901.01009049,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ])

In [176]:
# error rate
dif = np.dot(X_test, w_p_list) - y_test
np.sum(np.power(dif,2))

47592.671879579357